## importing libraries

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import string 
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from nltk.stem import WordNetLemmatizer
import re
from gensim.models import KeyedVectors
# import os
import pathlib

c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [2]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## Loading datasets 

In [3]:
dataset = pd.read_csv("train_sentiment.csv")
dataset.drop(dataset.columns[0] , axis=1, inplace=True)
dataset.iloc[1200]["review"]

"Ok,Badhiya h..,The attachments could have been a bit longer,This beater is less than half the price of regular branded ones but seems to do the job well.Not sure about heavy duty use but extremely handy for whisking eggs, ice cream, meringue, or light cake batters.This is the exact same beater that seems to be available under many names on this site with prices ranging between 430 to 599 (as of Dec 2022) so if buying pick the cheapest available price like I did.It doesn't come with any warranty, but I took a chance and seems to have paid off.,Ko,Have been using it for months and still it works smoothly. It's very easy to use and clean and is effective. Go for it,It's one of the most Convenient product and multiple use. It's durable and easy to use. I really like this product. 10/10.Must buy it.,Great product! Value for money. Go for it!"

## preprocessing

In [4]:
ls = stopwords.words('english')
regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
url_pattern = re.compile(r'https?://\S+|www\.\S+') # Removing urls from texts
# print(emoji_pattern.sub(r'', text)) # no emoji
def transform(s):
    s= s.lower()
    s = regrex_pattern.sub(r'',s) # removing emojis from strings
    s = url_pattern.sub(' ' , s)
    for item in string.punctuation:
        s = s.replace(item , " ")
    # no emoji
    
    # s = list(map(lambda x : "" if x in ls else x , s) )
    s = s.split()
    
    s = list(filter(lambda x : x not in ls , s))
    s = " ".join(s)
    return s
dataset["review"] = dataset["review"].apply(transform)  
# dataset["review"]  
final_dataset = dataset

    
    

In [5]:
def transformation(s): # Lemmatizing strings
    lemmatizer = WordNetLemmatizer()
    s = s.split()
    s = [lemmatizer.lemmatize(w) for w in s]
    return " ".join(s)
final_dataset["review"] = final_dataset["review"].apply(transformation)
# final_dataset
final_dataset["review"].iloc[1200]


'ok badhiya h attachment could bit longer beater le half price regular branded one seems job well sure heavy duty use extremely handy whisking egg ice cream meringue light cake batter exact beater seems available many name site price ranging 430 599 dec 2022 buying pick cheapest available price like come warranty took chance seems paid ko using month still work smoothly easy use clean effective go one convenient product multiple use durable easy use really like product 10 10 must buy great product value money go'

In [6]:
y = pd.DataFrame(final_dataset["rating"])
y.iloc[1279]["rating"] = 0
# for i in range(len(y)):
#     print(i)
#     float(y.iloc[i])
y["rating"] = y["rating"].apply(lambda x : 0 if float(x)<3 else 1) # changing labels into two classes
len(y[y["rating"]==0])
# final_dataset.drop(columns=["rating"] , inplace=True)

# y.dtypes

7

## TF-IDF

In [30]:
X = final_dataset["review"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X.values)

1110


In [8]:

X_train , X_test , y_train , y_test = train_test_split(X ,  y ,random_state=42 , train_size=0.8)    


## with logistic regression

In [9]:
model = LogisticRegression()
params = {"penalty": ['l1', 'l2'] ,
          "dual": [True , False] , 
          "fit_intercept": [True , False] ,
          "n_jobs": [5,10,15]
          }
grid = GridSearchCV(model , param_grid=params  , cv = 5)
# model.fit(X_train , y_train)
# y_pred = model.predict(X_test)
grid.fit(X_train , y_train)
y_pred = grid.predict(X_test)


f1_score(y_test, y_pred , average="weighted")
# grid = GridSearchCV(estimator=model ,param_grid=[{'penalty':['l1','l2']}, 
#               {'C':[1, 10, 100, 1000]}] , scoring="accuracy" , cv= 5)
# grid.fit(X_train , y_train)
# y_pred_new = grid.predict(X_test)
# f1_score(y_test, y_pred_new , average="weighted")

c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConve

0.6988411363879606

## with random forrest

In [10]:
model = RandomForestClassifier()
# params = {"n_estimators" : [20 , 50 , 100 , 150] , "criterion" :['gini', 'entropy', 'log_loss'] , "max_depth" : [2,3,4,5,] ,  }
params = {"n_estimators" : [50 ,100 ,150] , "criterion" :['gini', 'entropy', 'log_loss'] ,}
grid = GridSearchCV(estimator=model ,param_grid=params , cv = 5)
grid.fit(X_train , y_train)
y_pred =grid.predict(X_test)

# model.fit(X_train , y_train)
# y_pred =model.predict(X_test)
f1_score(y_test, y_pred , average="weighted")


c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionW

0.6988411363879606

## with knn

In [11]:
model = KNeighborsClassifier()
params = {"n_neighbors" : [2,3,4,],
          "weights": ['uniform', 'distance'],
          "algorithm" : ['auto', 'ball_tree', 'kd_tree', 'brute']}
grid = GridSearchCV(estimator=model , param_grid=params , cv = 5)
# model.fit(X_train , y_train)
# y_pred =model.predict(X_test)
grid.fit(X_train , y_train)
y_pred =grid.predict(X_test)
f1_score(y_test, y_pred , average="weighted")

c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: Data

0.7652188491707083

## Word2vec

In [12]:
'''
for running this block properly You have to download the GoogleNews-vectors-negative300.bin file .
due to its large size I didnt put it in the project folder !!!!


'''
# # # Path to the downloaded model
# # print(pathlib.Path().resolve())
curr = str(pathlib.Path().resolve()) +'\GoogleNews-vectors-negative300.bin'
# print(curr)
# model_path = 'C:\Users\ma-na\OneDrive\Desktop\data minding final project\GoogleNews-vectors-negative300.bin'
# # # Load the model
word2vec = KeyedVectors.load_word2vec_format(curr, binary=True)
def vectorizing(x):
    x = x.split()
    if x:
        vector = np.mean([word2vec[item] for item in x if item in word2vec])
    else:
        vector = np.zeros(word2vec.vector_size)
    return vector
X = dataset["review"].apply(vectorizing)

# # Example usage
# print(vector)
# TODO : complete this part


In [13]:
X = pd.DataFrame(X)

In [14]:
X_train , X_test , y_train , y_test = train_test_split(X ,  y ,random_state=42 , train_size=0.8)    
X.shape

# vector

(1465, 1)

## with logistic regression

In [15]:
model = LogisticRegression()
params = {"penalty": ['l1', 'l2' , None] ,
        #   "dual": [True , False] , 
        #   "fit_intercept": [True , False] ,
          "n_jobs": [-1]
          }
grid = GridSearchCV(model , param_grid=params  , cv = 5)
# model.fit(X_train , y_train)
# y_pred = model.predict(X_test)
grid.fit(X_train , y_train)
y_pred = grid.predict(X_test)
# model.fit(X_train , y_train)
# y_pred = model.predict(X_test)
f1_score(y_test, y_pred , average="weighted")

c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConve

0.6394117960750854

## with random forrest

In [16]:
model = RandomForestClassifier()
params = {"n_estimators" : [50 ,100 ,150] , "criterion" :['gini', 'entropy', 'log_loss'] ,}
grid = GridSearchCV(estimator=model ,param_grid=params , cv = 5)
grid.fit(X_train , y_train)
y_pred =grid.predict(X_test)
# model.fit(X_train , y_train)
# y_pred =model.predict(X_test)
f1_score(y_test, y_pred , average="weighted")

c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionW

0.6894698914345858

## with knn

In [17]:
model = KNeighborsClassifier()
params = {"n_neighbors" : [3,5,9,15],
          "weights": ['uniform', 'distance'],
        #   "algorithm" : ['auto', 'ball_tree', 'kd_tree', 'brute']
          }
grid = GridSearchCV(estimator=model , param_grid=params , cv = 5)
# model.fit(X_train , y_train)
# y_pred =model.predict(X_test)
grid.fit(X_train , y_train)
y_pred =grid.predict(X_test)
# model.fit(X_train , y_train)
# y_pred =model.predict(X_test)
f1_score(y_test, y_pred , average="weighted")

c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: Data

0.6720600047622828

## Bert

In [18]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
X = model.encode(final_dataset["review"])
X[0]

array([-6.69318438e-01,  7.00509176e-02, -3.19714807e-02, -2.67025065e-02,
        2.44630687e-02, -2.48685688e-01,  2.83953119e-02,  2.24959388e-01,
       -6.42212272e-01,  1.29354730e-01,  1.11214377e-01,  1.98740423e-01,
       -3.88126880e-01,  3.22367936e-01, -1.73885658e-01,  5.53371198e-02,
        2.49352351e-01, -8.17436874e-02,  3.52855086e-01,  3.12721938e-01,
       -5.26564479e-01, -1.72271654e-01, -8.19627289e-03,  2.32079439e-02,
       -4.73086447e-01,  2.55884141e-01,  1.33054912e-01, -7.92992190e-02,
        7.47192204e-01,  1.40486285e-01,  3.22642744e-01,  3.27143401e-01,
       -2.19570026e-01,  1.35750994e-01, -4.00566548e-01,  1.45926014e-01,
       -1.24740794e-01, -1.69441193e-01,  1.72501039e-02, -4.58614647e-01,
       -6.70627803e-02,  2.27651939e-01,  1.13650225e-01, -8.87177736e-02,
        1.85035646e-01,  3.79543215e-01,  2.47814089e-01,  1.77905783e-01,
        2.73748755e-01, -8.72481540e-02,  1.33270308e-01,  1.71098456e-01,
        2.53664136e-01, -

In [19]:
X_train , X_test , y_train , y_test = train_test_split(X ,  y ,random_state=42 , train_size=0.8)    
X.shape


(1465, 384)

## with logistic regression

In [20]:
model = LogisticRegression()
params = {"penalty": ['l1', 'l2' , None] ,
        #   "dual": [True , False] , 
        #   "fit_intercept": [True , False] ,
          "n_jobs": [-1]
          }
grid = GridSearchCV(model , param_grid=params  , cv = 5)
# model.fit(X_train , y_train)
# y_pred = model.predict(X_test)
grid.fit(X_train , y_train)
y_pred = grid.predict(X_test)
# model.fit(X_train , y_train)
# y_pred = model.predict(X_test)
f1_score(y_test, y_pred , average="weighted")

c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConve

0.7109739751377977

## with random forrest

In [21]:
model = RandomForestClassifier()
params = {"n_estimators" : [50 ,100 ,150] , "criterion" :['gini', 'entropy', 'log_loss'] ,}
grid = GridSearchCV(estimator=model ,param_grid=params , cv = 5)
grid.fit(X_train , y_train)
y_pred =grid.predict(X_test)
# model.fit(X_train , y_train)
# y_pred =model.predict(X_test)
f1_score(y_test, y_pred , average="weighted")

c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionW

KeyboardInterrupt: 

## with knn

In [137]:
model = KNeighborsClassifier()
params = {"n_neighbors" : [3,5,9,15],
          "weights": ['uniform', 'distance'],
        #   "algorithm" : ['auto', 'ball_tree', 'kd_tree', 'brute']
          }
grid = GridSearchCV(estimator=model , param_grid=params , cv = 5)
# model.fit(X_train , y_train)
# y_pred =model.predict(X_test)
grid.fit(X_train , y_train)
y_pred =grid.predict(X_test)
# model.fit(X_train , y_train)
# y_pred =model.predict(X_test)
f1_score(y_test, y_pred , average="weighted")

c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\ma-na\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:228: Data

0.9948834631428488